# Snapping Points to Road Networks

This notebook provides several examples on how points can be snapped to road networks. It is possible to snap mapillary points to either open street map segments, or any user supplied network topology. If a user-supplied topology is given, then it must be in the form of a geopandas dataframe with a `LineString` geometry column.

In [1]:
# note, to load the local module, it might be necessary to add the repository directory to the sys path:
import sys
sys.path.append("/Users/iosefa/repos/misc/SU_GCPsystem/")

In [ ]:
from src.controller import MapillaryImage
from src.utils import DataUtils
from src.visualizer import Visualize

After loading the necessary libraries for these examples, we will set the following required environmental variables. Please replace the values within `<>` with the appropriate values.

In [ ]:
%env DATABASE_URL=<DATABASE_URL>
%env SERVICE_ACCOUNT_PRIVATE_KEY=<PATH_TO_PRIVATE_KEY>

## Creating and Saving Snapped Geometries

First, we will create snapped geometries and save these records to the database. The first step is to initialize the `MappilaryImage` class. This will open the connection to the database. Then, we can query some data to work with. In the following examples, we will query by image ID.

In [ ]:
imgs = MapillaryImage()
gdf = imgs.select_by_image_id(804124637170637)

We can map this using the `Visualize` class:

In [ ]:
vis = Visualize(gdf)
vis.map()

The `DataUtils` class provides functions to snap points to road networks using either OSM or a user-supplied network topology. These functions also provide options to write these points to the database and how to handle conflicts. Thus, to snap points, we need to initialize the `DataUtils` class with the data that we want to snap:

In [ ]:
dutil = DataUtils(gdf)

### Snapping to an OSM Network

To snap to an OSM network, we only need to call the `snap_to_road_network` function and set `use_osm=True`.

In [ ]:
gdf_osm = dutil.snap_to_road_network(100, gdf, update_db=False, use_osm=True)

This will snap the point to the nearest OSM road within 100m. By default, when snapping to OSM, it will incluse all network types (roads, paths, etc), including private ones.

The returned geodataframe only includes the related image_id, a flag to record if OSM was used, and the geometry. To visualize this, we will need to merge this data with the original dataframe. We can also map both the snapped geometry and the original geometry using the `additional_geometries` parameter od the `map` function.

In [ ]:
merged = gdf[['id', 'seq', 'camera_type', 'image_url', 'geometry']].merge(gdf_osm.rename({'image_id': 'id', 'geometry': 'osm_geometry'}, axis=1), left_on='id', right_on='id', how='right')

vis = Visualize(merged)
vis.map(additional_geometries=['osm_geometry'])

### Snapping to a Provided Network

To snap the mapillary points to a user-supplied network, we first need to create a geodataframe that includes these gemoetries. The following reads all shapefiles in a given directory. Then, we can snap to the road network as above and visualize and compare with the osm snapped results.

In [ ]:
import geopandas as gp
import os

roads = None
shp_dir = "/Users/iosefa/downloads/roads/"
for file in os.listdir(shp_dir):
    if file.endswith(".shp"):
        tmp = gp.read_file(os.path.join(shp_dir, file)).to_crs('EPSG:4326')
        roads = gp.pd.concat([roads, tmp])

In [ ]:
gdf_snapped = dutil.snap_to_road_network(100, gdf, update_db=False, use_osm=False, network=roads)
merged = gdf[['id', 'seq', 'camera_type', 'image_url', 'geometry']].merge(gdf_snapped.rename({'image_id': 'id', 'geometry': 'snapped_geometry'}, axis=1), left_on='id', right_on='id', how='right').merge(gdf_osm.rename({'image_id': 'id', 'geometry': 'osm_geometry'}, axis=1), left_on='id', right_on='id', how='right')

vis = Visualize(merged)
vis.map(additional_geometries=['snapped_geometry', 'osm_geometry'])

### Saving and Loading Snapped Geometries to the Database

When creating a snapped dataframe, it is also possible to save the resulting geometries to the database, which can then be queried later without the need to re-snap the geometries. Setting the `update_db` parameter to `True` will safe the resulting dataframe to the database and setting `update_conflicting=True` will tell the function to overwrite any existing snapped image_id geometries. This may be useful when changing the snapped to geometry to use a user supplied road network instead of OSM, for example.

In [ ]:
dutil.snap_to_road_network(100, gdf, update_db=True, use_osm=True, update_conflicting=True)

When we query data, we can choose to include snapped geometries in the returned table using the `include_snapped` parameter:

In [ ]:
imgs.select_by_image_id(804124637170637, include_snapped=True)

Now, should we decide to use our own road network and update the database with it, we would:

In [ ]:
dutil.snap_to_road_network(100, gdf, update_db=True, use_osm=False, network=roads, update_conflicting=True)

Querying the database, we can see that the snapped geometry was updated and we can visualize all three geometries: default, computed, and snapped:

In [ ]:
gdf = imgs.select_by_image_id(804124637170637, include_snapped=True)
vis = Visualize(gdf)
vis.map(additional_geometries=['geometry', 'computed_geometry', 'snapped_geometry'])